In [2]:
# from vit_jax import models
# import jax
# import jax.numpy as jnp
from PIL import Image
import numpy as np
import glob
import os
import numpy as np
import os
import pandas as pd
import glob
import json
import clip
import torch


import googletrans
import translate


In [3]:
# Edit

mode = "clip"
root_path = "/home/toonies/AI-Challenge/AI_Challenge"

# Đường dẫn file text (đã tồn tại)
ocr_inf = root_path + "data/OCR_ASR/info_ocr_loc.txt"


# Đường dẫn để lưu file npy (chưa tồn tại)
ocr_save_np = f"data/models/npy_{mode}_ocr"

In [4]:
class Translation:
    def __init__(self, from_lang='vi', to_lang='en', mode='googletrans'):
        # The class Translation is a wrapper for the two translation libraries, googletrans and translate. 
        self.__mode = mode
        self.__from_lang = from_lang
        self.__to_lang = to_lang

        if mode in 'googletrans':
            self.translator = googletrans.Translator()
        elif mode in 'translate':
            self.translator = translate.Translator(from_lang=from_lang,to_lang=to_lang)

    def preprocessing(self, text):
        """
        It takes a string as input, and returns a string with all the letters in lowercase
        :param text: The text to be processed
        :return: The text is being returned in lowercase.
        """
        return text.lower()

    def __call__(self, text):
        """
        The function takes in a text and preprocesses it before translation
        :param text: The text to be translated
        :return: The translated text.
        """
        text = self.preprocessing(text)
        return self.translator.translate(text) if self.__mode in 'translate' \
                else self.translator.translate(text, dest=self.__to_lang).text
    
translate = Translation()

# Check có hàng trống không và xóa

In [5]:
###################### Nếu có thì sửa lại code (code hiện k xuất dc nội dung ocr) ###################### 

ocr_inf = root_path+"/data/dicts/info_ocr.txt"
df_ocr = pd.read_csv(ocr_inf, delimiter=",", header=None)
print(len(df_ocr))
for i in range(len(df_ocr[2])):
    if df_ocr[2][i]== " " or df_ocr[2][i]== "":
        print("Index",i)
        df_ocr = df_ocr.drop(index=i)
print(len(df_ocr))

# output_file = root_path+"/data/dicts/cleaned_info_ocr.txt"
# with open(output_file, 'w') as f:
#      f.write(df_ocr.to_string(index=False))
# print("Saved: ", output_file)

1114
1114


In [6]:
if mode == "lit":
    model_name = 'LiT-B16B'
    lit_model = models.get_model(model_name)
    lit_variables = lit_model.load_variables()
    tokenizer = lit_model.get_tokenizer()
    image_preprocessing = lit_model.get_image_preprocessing()
    pp = lit_model.get_pp()
    translate = Translation()
    os.system('TF_CPP_MIN_LOG_LEVEL=0')
elif mode == "clip":
    device = "cuda"
    print(device)
    model, preprocess = clip.load("ViT-B/16", device=device)

cuda


# Create npy file

In [7]:
df_ocr = pd.read_csv(ocr_inf, delimiter=",", header=None)
root_dicts = root_path+"data/dicts/"
index = 0
if mode == "lit":
    for path in df_ocr[0].unique():
        os.makedirs(f"{root_dicts}/{path}", exist_ok = True)
        re_feats = []
        for i in range(len(df_ocr.loc[df_ocr[0] == path])):
            text = translate(df_ocr[2][index]) # anhasd asada s
            text = [translate(text),]
            print(f"Idx: {index},{path}, {i}")
            tokens = tokenizer([text])
            _, text_embeddings, _ = lit_model.apply(lit_variables, tokens=tokens)

            index += 1
            re_feats.append(text_embeddings)
        outfile = f'{ocr_save_np}/{path}.npy'
        np.save(outfile, re_feats)
        print(f"Save {outfile}")
    print("num of index ", index)

elif mode == "clip":
    max_len = 0
    for path in df_ocr[0].unique():
        os.makedirs(f"{root_dicts}/{path}", exist_ok = True)
        re_feats = []
        for i in range(len(df_ocr.loc[df_ocr[0] == path])):
            text = translate(df_ocr[2][index])
            if len(text) >= max_len:
                max_len = len(text)
            print("max len of str is ",max_len)
            text = clip.tokenize([text]).to(device)
            with torch.no_grad():
                text_features = model.encode_text(text)
            text_features = text_features.cpu().detach().numpy().astype(np.float32)
            index += 1
            re_feats.append(text_features)
        outfile = f'{ocr_save_np}/{path}.npy'
        np.save(outfile, re_feats)
        print(f"Save {outfile}")
    print("num of index ", index)

max len of str is  23
max len of str is  23
max len of str is  40
max len of str is  40
max len of str is  113
max len of str is  113
max len of str is  113
max len of str is  199
max len of str is  199
max len of str is  199
max len of str is  199
max len of str is  199
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str is  268
max len of str 

RuntimeError: Input 179 HT HD 06:32:05 Genisody Nick HVZZ MO1 Dingle Recreeningly Coang Angnisms HIV HIV LBN Mot Adixal Jounge Thiot Chivs Wt Test HIV Self HIV Monthly Y Mylan HIV HIV HIV Nighiem he is Deputy Governor General Satra Growing Soly Passionate Oil Hà Ngoc Son.At risk The Khangi is too long for context length 77

# Create bin file

In [56]:
import numpy as np
import faiss
import glob
import json
import matplotlib.pyplot as plt
import os
import math
import torch
import pandas as pd
import re
import time

from langdetect import detect
def process_name(name: int):
    return "0"*(6-len(str(name))) + str(name)

def write_bin_file_ocr(bin_path: str, root_path: str, method='cosine', feature_shape= 768): # Edit 512, 768
  idx = 0
  id = 0
  if method in 'L2':
    index = faiss.IndexFlatL2(feature_shape)
  elif method in 'cosine':
    index = faiss.IndexFlatIP(feature_shape)
  else:
    assert f"{method} not supported"
  
  for _ in range(len(df_ocr[1])):
    
    # Create image path "*/*/*.jpg"
    image_path = f"{root_path}/{df_ocr[0][idx]}/{process_name(df_ocr[1][idx])}.jpg" 

    # Call *.npy path that was created
    video_name = f'{df_ocr[0][idx]}.npy'
    LIT_name = f"data/models/npy_ocr" # Edit 
    feat_path = os.path.join(LIT_name, video_name) 

    # Load npy file
    feats = np.load(feat_path)
    ids = os.listdir(re.sub('/\d+.jpg','',image_path))
    ids = sorted(ids, key=lambda x:int(x.split('.')[0]))

########################################################################
 # Output ID 1, 2, 3, 4 . if len path == id reset id
    feat = feats[id]
    id = 0 if len(df_ocr.loc[df_ocr[0] == df_ocr[0][1048]]) == (id + 1) else id + 1

    print("ID: ", id)

########################################################################

    
    # print(feat.shape)
    feat = feat.astype(np.float32).reshape(1,-1)
    # print("##########################################")
    # print(" Feat after reshape: ", feat.shape)
    index.add(feat)
    
    idx +=1

  faiss.write_index(index, os.path.join(bin_path, f"faiss_LIT_OCR_{method}.bin"))

  print(f'Saved {os.path.join(bin_path, f"faiss_LIT_OCR_{method}.bin")}')
  print(f"Number of Index: {idx}")

In [ ]:
write_bin_file_ocr(bin_path = "data/models", root_path = "data/KeyFramesC00_V00/")

# Load and search with bin file

In [64]:
from utils.translate_processing import Translation

class MyFaiss:
  def __init__(self, bin_file: str, mode = "lit"):    
    self.index = self.load_bin_file(bin_file)
    self.translate = Translation()
    ocr_inf = "data/OCR_ASR/info_ocr_loc.txt"
    self.df_ocr = pd.read_csv(ocr_inf, delimiter=",", header=None)
    if mode == "lit":
      os.system('TF_CPP_MIN_LOG_LEVEL=0')
      self.lit_model = models.get_model("LiT-B16B")
      self.lit_var = self.lit_model.load_variables()
      self.tokenizer = self.lit_model.get_tokenizer()
      
  def load_bin_file(self, bin_file: str):
    return faiss.read_index(bin_file)
  
  def text_search(self, text, k):
    if detect(text) == 'vi':
      text = self.translate(text)
    text = self.translate(text)
    print("Text translation: ", text)
    tokens = self.tokenizer([text])
    _, text_features, _ = self.lit_model.apply(self.lit_var, tokens=tokens)
    scores, idx_image = self.index.search(np.array(text_features), k=k)
    idx_image = idx_image.flatten()
    print("idx_img ",idx_image)
    return idx_image

In [69]:
def main():
    faiss_search = MyFaiss(bin_file='data/models/faiss_LIT_OCR_cosine.bin')
    text = "tphcm trẻ mắc bệnh tay chân miệng tăng gấp lần trong một tháng"
    idx_image = faiss_search.text_search(text, k=9)

    
if __name__ == "__main__":
    main()



⚠️ Reusing local copy: LiT-B16B.npz
Text translation:  ho chi minh city young people with hand, foot and mouth disease increased more than a month
idx_img  [224 220 218 221 223 222 404 542 253]


# Test  function

In [61]:
from vit_jax import models
from utils.translate_processing import Translation
import numpy as np
import faiss
import pandas as pd
import re
import os
from langdetect import detect
import time

def time_complexity(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        results = func(*args, **kwargs)
        print('Time inference of {} and ocr: {}'.format(args[0].mode, time.time() - start))
        return results
    return wrapper

class Faiss_OCR():
    def __init__(self, bin_file: str, info_file: str , root_img:str, mode = "lit"):
        self.df_ocr = pd.read_csv(info_file, delimiter=",", header=None)
        self.translate = Translation()
        self.index = self.load_bin_file(bin_file)
        self.root_img = root_img
        self.mode = mode

        if mode == "lit":
            os.system('TF_CPP_MIN_LOG_LEVEL=0')
            self.lit_model = models.get_model("LiT-B16B")
            self.lit_var = self.lit_model.load_variables()
            self.tokenizer = self.lit_model.get_tokenizer()
        else:
            pass

    def process_name_img(self, name: int):
        return "0"*(6-len(str(name))) + str(name)

    def convert_idx_to_path(self,idx_image):
        path_arr = []
        for idx in idx_image:
            path = f"{self.root_img}/{self.df_ocr[0][idx]}/{self.process_name_img(self.df_ocr[1][idx])}.jpg"
            path_arr.append(path)
        return path_arr

    def load_bin_file(self, bin_file):
        return faiss.read_index(bin_file)
    
    @time_complexity
    def text_search(self, text, k = 9):
        if detect(text) == 'vi':
            text = self.translate(text)
        text = self.translate(text)
        print("Text translation: ", text)
        tokens = self.tokenizer([text])
        _, text_features, _ = self.lit_model.apply(self.lit_var, tokens=tokens)
        scores, idx_image = self.index.search(np.array(text_features), k=k)
        idx_image = idx_image.flatten()
        # arr_path = self.convert_idx_to_path(idx_image)
        arr_paths = [self.df_ocr[1][id_image] for id_image in idx_image]
        return arr_paths
    
def main():
    faiss_search = Faiss_OCR(bin_file='data/models/faiss_LIT_OCR_cosine.bin', info_file = "data/OCR_ASR/info_ocr_loc.txt", root_img = "data/KeyFramesC00_V00")
    text = "himship trên tỷ đồng tôn tạo cảnh quan di tích"
    idx_image = faiss_search.text_search(text, k=9)
    print(idx_image)

if __name__ == "__main__":
    main()



⚠️ Reusing local copy: LiT-B16B.npz
Text translation:  himship over billion dong to embellish the landscape of the monument
Time inference of lit and ocr: 0.9316995143890381
[11192, 11209, 19158, 10995, 19119, 11246, 14650, 14591, 19257]


# Update txt file

In [34]:
def process_name_img( name: int):
    return "0"*(6-len(str(name))) + str(name)

def convert_idx_to_path(idx_image):
    return f"KeyFramesC00_V00/{df_ocr[0][idx_image]}/{process_name_img(df_ocr[1][idx_image])}.jpg"
     

In [ ]:
df_ocr = pd.read_csv("data/OCR_ASR/info_ocr_loc.txt", delimiter=",", header=None)
file_path = "data/OCR_ASR/update_info_ocr.txt"
items = []
for i in range(len(df_ocr)):
    item = convert_idx_to_path(i)
    items.append(item)

with open(file_path, "w") as file:
    for i in items:
        file.write("%s\n" %i)
        print(i)

# Update OCR function

In [1]:



from vit_jax import models
from utils.translate_processing import Translation
import numpy as np
import faiss
import pandas as pd
import re
import os
from langdetect import detect
import time

def time_complexity(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        results = func(*args, **kwargs)
        print('Time inference of {} and ocr: {}'.format(args[0].mode, time.time() - start))
        return results
    return wrapper

class Faiss_OCR():
    def __init__(self, bin_file: str, info_file: str , root_img:str, mode = "lit"):
        self.df_ocr = pd.read_csv(info_file, delimiter=",", header=None)
        self.translate = Translation()
        self.index = self.load_bin_file(bin_file)
        self.root_img = root_img
        self.mode = mode

        if mode == "lit":
            os.system('TF_CPP_MIN_LOG_LEVEL=0')
            self.lit_model = models.get_model("LiT-B16B")
            self.lit_var = self.lit_model.load_variables()
            self.tokenizer = self.lit_model.get_tokenizer()
        else:
            pass



    def load_bin_file(self, bin_file):
        return faiss.read_index(bin_file)
    
    @time_complexity
    def text_search(self, text, k = 9):
        if detect(text) == 'vi':
            text = self.translate(text)
        text = self.translate(text)
        print("Text translation: ", text)
        tokens = self.tokenizer([text])
        _, text_features, _ = self.lit_model.apply(self.lit_var, tokens=tokens)
        scores, idx_image = self.index.search(np.array(text_features), k=k)
        idx_image = idx_image.flatten()
        return [self.root_img+self.df_ocr[0][id_image] for id_image in idx_image], scores
    
def main():
    faiss_search = Faiss_OCR(bin_file='data/models/faiss_LIT_OCR_cosine.bin', info_file = "data/OCR_ASR/update_info_ocr.txt", root_img = "data/")
    text = "himship trên tỷ đồng tôn tạo cảnh quan di tích"
    idx_image, scores = faiss_search.text_search(text, k=9)
    print(scores)

if __name__ == "__main__":
    main()

2023-08-22 15:12:46.085322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



⚠️ Reusing local copy: LiT-B16B.npz


2023-08-22 15:12:49.589474: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-22 15:12:49.656316: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Text translation:  himship over billion dong to embellish the landscape of the monument
Time inference of lit and ocr: 3.221402645111084
[[0.8692388  0.8692388  0.85251015 0.850931   0.8483517  0.8375005
  0.8212406  0.8178458  0.8167915 ]]
